# Message Memory in Agent backed by a database

This notebook goes over adding memory to an Agent where the memory uses an external message store. Before going through this notebook, please walkthrough the following notebooks, as this will build on top of both of them:

- [Memory in LLMChain](/docs/modules/memory/adding_memory)
- [Custom Agents](/docs/modules/agents/how_to/custom_agent)
- [Memory in Agent](/docs/modules/memory/agent_with_memory)

In order to add a memory with an external message store to an agent we are going to do the following steps:

1. We are going to create a `RedisChatMessageHistory` to connect to an external database to store the messages in.
2. We are going to create an `LLMChain` using that chat history as memory.
3. We are going to use that `LLMChain` to create a custom Agent.

For the purposes of this exercise, we are going to create a simple custom Agent that has access to a search tool and utilizes the `ConversationBufferMemory` class.

In [70]:
from langchain import hub
from langchain.agents import AgentExecutor, Tool
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_openai import OpenAI
import os

In [71]:
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"
os.environ["GOOGLE_CSE_ID"] = "GOOGLE_CSE_ID"
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
    
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    )
]

Notice the usage of the `chat_history` variable in the `PromptTemplate`, which matches up with the dynamic key name in the `ConversationBufferMemory`.

In [72]:
prompt = hub.pull("hwchase17/react")

Now we can create the `RedisChatMessageHistory` backed by the database.

In [73]:
message_history = RedisChatMessageHistory(
    url="redis://127.0.0.1:6379/0", ttl=600, session_id="my-session"
)

We can now construct the `LLMChain`, with the Memory object, and then create the agent.

In [74]:
from langchain.agents import create_react_agent

model = OpenAI()
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [75]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

agent_with_chat_history.invoke({"input": "How many people live in canada?"},config={"configurable": {"session_id": "<foo>"}})

{'input': 'How many people live in canada?',
 'chat_history': [],
 'output': "The average population of Canada is 39,032,811 as of Tuesday, April 9, 2024, based on Worldometer elaboration of the latest United Nations data 1. · Canada 2023\xa0... It is a sparsely inhabited country of 40 million people, the vast majority residing south of the 55th parallel in urban areas. Canada's capital is Ottawa and\xa0... Index to the latest information from the Census of Population. This survey conducted by Statistics Canada provides a statistical portrait of Canada and its\xa0... Mar 12, 2024 ... Indicators, data and detailed analysis related to Canadian demographics, from a wide set of data sources including population estimates and\xa0... total: 35.34 million (2021 est.) percent of population: 93% (2021 est.) comparison ranking: total 28 · Broadband - fixed subscriptions. total:\xa0... Quarterly population estimate - Canada (Note) (January 1, 2024). 40,769,890. 0.6% increase · Consumer Price Inde

To test the memory of this agent, we can ask a followup question that relies on information in the previous exchange to be answered correctly.

In [76]:
agent_with_chat_history.invoke({"input": "what is their national anthem called?"},config={"configurable": {"session_id": "<foo>"}})

{'input': 'what is their national anthem called?',
 'chat_history': [HumanMessage(content='How many people live in canada?'),
  AIMessage(content="The average population of Canada is 39,032,811 as of Tuesday, April 9, 2024, based on Worldometer elaboration of the latest United Nations data 1. · Canada 2023\xa0... It is a sparsely inhabited country of 40 million people, the vast majority residing south of the 55th parallel in urban areas. Canada's capital is Ottawa and\xa0... Index to the latest information from the Census of Population. This survey conducted by Statistics Canada provides a statistical portrait of Canada and its\xa0... Mar 12, 2024 ... Indicators, data and detailed analysis related to Canadian demographics, from a wide set of data sources including population estimates and\xa0... total: 35.34 million (2021 est.) percent of population: 93% (2021 est.) comparison ranking: total 28 · Broadband - fixed subscriptions. total:\xa0... Quarterly population estimate - Canada (Not

We can see that the agent remembered that the previous question was about Canada, and properly asked Google Search what the name of Canada's national anthem was.

For fun, let's compare this to an agent that does NOT have memory.

In [77]:
agent = create_react_agent(model, tools, prompt)
agent_executor__without_memory = AgentExecutor(agent=agent, tools=tools)

In [78]:
agent_executor__without_memory.invoke({"input": "How many people live in canada?"})

{'input': 'How many people live in canada?',
 'output': 'The estimated population of Canada in 2022 is approximately 39 million people.'}

In [81]:
agent_executor__without_memory.invoke({"input": "what is their national anthem called?"})

{'input': 'what is their national anthem called?',
 'output': 'The national anthem of a country can be found by searching "national anthem [country name]."'}